#Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields

## Introduction

The estimation of human posture is the process of modeling the human body as a set of some parts (e.g., arms, legs, torso, etc.) and finding their positions in an image. Several studies have been developed in order to generate these estimates in real time using various techniques of Deep Learning. In this Notebook we study the development done in [[1]](https://ieeexplore-ieee-org.ezproxy.imt-atlantique.fr/document/8099626), which makes use of Part Affinity Fields for this purpose.

Inferring the pose of multiple people in images, especially socially engaged individuals, presents a unique set of challenges. First, each image may contain an unknown number of people that can occur at any position or scale. Second, interactions between people induce complex spatial interference, due to contact, occlusion, and limb articulations, making association of parts difficult. Third, runtime complexity tends to grow with the number of people in the image, making realtime performance a challenge.

In [[1]](https://ieeexplore-ieee-org.ezproxy.imt-atlantique.fr/document/8099626), a method for estimating the posture of various persons at multiple public reference points is presented. It uses the ascending representation of association scores through the part affinity fields (PAF), a set of 2D vector fields that encode the location and orientation of the limbs over the image domain. These bottom-up representations of detection and association are simultaneously inferred and encode the global context of the scene.

First, we will download the repository that we copied from the original repository in order to make some changes for educational purposes. Then, we install libraries and some dependences explained in the original repository.

In [3]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
colab_path = "/content/gdrive/My Drive/Colab Notebooks/"

# Independence install
!sudo apt-get install swig
%cd $colab_path
# Repository clone
!git clone https://github.com/Johansmm/RT-multiperson-pose-pytorch.git
%cd "RT-multiperson-pose-pytorch"
# Libraries install
!python -m pip install -U -r ./requirements.txt
!git submodule init && git submodule update
%cd lib/pafprocess 
# Repository compile
!sh make.sh
%cd $colab_path"/RT-multiperson-pose-pytorch"
# Weights download
if not os.path.isfile("./pose_model.pth"):
  !wget https://www.dropbox.com/s/ae071mfm2qoyc8v/pose_model.pth


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
/content/gdrive/My Drive/Colab Notebooks
fatal: destination path 'RT-multiperson-pose-pytorch' already exists and is not an empty directory.
/content/gdrive/My Drive/Colab Notebooks/RT-multiperson-pose-pytorch
Requirement already up-to-date: Cython in /usr/local/lib/python3.6/dist-packages (from -r ./requirements.txt (line 1)) (0.29.21)
Requirement already up-to-date: torch>=1.2 in /usr/local/lib/python3.6/dist-packages (from -r ./requirements.txt (line 2)) (1.7.0)
Requirement already up-to-date: torchvision>=0.4.0 in /usr/local/lib/python3.6/dist-packages (from -r ./requirements.txt (line 3)) (0.8.1)
Requirement already up-to-date: progress in /usr/loca

In [8]:
%cd $colab_path"/RT-multiperson-pose-pytorch"
!python "./demo/picture_demo.py"

/content/gdrive/My Drive/Colab Notebooks/RT-multiperson-pose-pytorch
Bulding VGG19
/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
Traceback (most recent call last):
  File "./demo/picture_demo.py", line 47, in <module>
    model = torch.nn.DataParallel(model).cuda()
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 463, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 359, in _apply